In [1]:
# python packages pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import os
import inspect

import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Bidirectional, Activation
from keras.layers import CuDNNLSTM
from keras.utils.np_utils import to_categorical
# from keras.callbacks import EarlyStopping
from keras.layers import Dropout

from sklearn.model_selection import train_test_split
import importlib
import utilis

# custom

Using TensorFlow backend.


In [19]:
sys.path.append("..")

In [26]:
# custom python scripts
from packages import generator

In [27]:
importlib.reload(generator)

<module 'packages.generator' from '../packages/generator.py'>

In [28]:
# # check version
# print(inspect.getsource(generator.Keras_DataGenerator))

# Bidirectional LSTM with Hypotheses

In [11]:
# Check that you are running GPU's
utilis.GPU_checker()





You are runnning an instance with 4 GPU's


In [12]:
utilis.aws_setup()

AWS SETUP SHOULD BE COMPLETE, we are on <botocore.client.S3 object at 0x7fe4c02b3048>


# Config, generators and train

In [29]:
INPUT_TENSOR_NAME = "inputs_input"
SIGNATURE_NAME = "serving_default"
W_HYP = True
LEARNING_RATE = 0.001
BATCH_SIZE = 64

# constnats
VOCAB_SIZE = 1254
INPUT_LENGTH = 3000 if W_HYP else 1000
EMBEDDING_DIM = 128

print(INPUT_LENGTH)

3000


In [30]:
importlib.reload(generator)

# generators
training_generator = generator.Keras_DataGenerator(data_dir='',  dataset='train_new', w_hyp=W_HYP)
print()
validation_generator = generator.Keras_DataGenerator(data_dir='', dataset='valid_new', w_hyp=W_HYP)

Input data_dir is:  
# of batches:  5888.0
# of partitions:  92.0
Retrieving data from;  deephol-data-processed/proofs/human/train_new/
Generating examples from a set of 376832 examples

Input data_dir is:  
# of batches:  1600.0
# of partitions:  25.0
Retrieving data from;  deephol-data-processed/proofs/human/valid_new/
Generating examples from a set of 102400 examples


### testing generator

In [33]:
# model
def build_model(vocab_size, embedding_dim, input_length):
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_dim, input_length=input_length))
    model.add(SpatialDropout1D(0.4))
    model.add(Bidirectional(CuDNNLSTM(128)))
    model.add(Dropout(0.4))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(41, activation='softmax'))
    return model

model = build_model(VOCAB_SIZE, EMBEDDING_DIM, INPUT_LENGTH)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 3000, 128)         160512    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 3000, 128)         0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256)               264192    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________

In [34]:
## ARE YOU LOADING A MODEL IF YES RUN TEH FOLLOWING LINES 
# from keras.models import model_from_json
# json_file = open('model.json', 'r')
# loaded_model_json = json_file.read()
# json_file.close()
# loaded_model = model_from_json(loaded_model_json)
# # load weights into new model
# loaded_model.load_weights("model.h5")
# print("Loaded model from disk")
# # REMEMEBER TO COMPILE 
# loaded_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [35]:
#overwriting model
# model = loaded_model

In [ ]:
%%time
#try and make it run until 9 am GMT+1
n_epochs = 2
history = model.fit_generator(generator=training_generator,
                            validation_data=validation_generator,
                            verbose=1,
                            use_multiprocessing=True,
                            epochs=n_epochs)

Epoch 1/2
  24/5888 [..............................] - ETA: 6:13:34 - loss: 2.7369 - acc: 0.1534

## Save modek

In [14]:
# FOR SAVING MODEL
model_json = model_GPU.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [15]:
#WARNING_DECIDE_HOW_TO_NAME_LOG
#descriptionofmodel_personwhostartsrun
#e.g. LSTM_128encoder_etc_tanc
LOSS_FILE_NAME = "forjeff"

#WARNING NUMBER 2 - CURRENTLY EVERYTIME YOU RERUN THE CELLS BELOW THE FILES WITH THOSE NAMES GET WRITTEN OVER

In [16]:
# save history - WARNING FILE NAME 
utilis.history_saver_bad(history, LOSS_FILE_NAME)

SAVED SOME LOGS -- OVERWROTE OLD LOGS -- SOMEONE NEEDS TO FIX THIS
